<img src="logo.png" width="420px">

# `TODOIST AUTOMATION`

### Settings

#### Libraries

In [ ]:
from todoist_api_python.api import TodoistAPI
from datetime import datetime, timedelta, timezone
import functions as fun
import random

#### Dependences

In [ ]:
import credentials

#### API

In [ ]:
api_token = credentials.api_token
api = TodoistAPI(api_token)
headers = {
    "Content-Type": "application/json",
    "X-Request-Id": "un_valor_único_por_cada_solicitud",
    "Authorization": f"Bearer {api_token}"
}

### `Data`

#### Projects

In [ ]:
all_projects = fun.getProjects()
projects_dict_id, projects_dict_name = fun.getProjectsDicts(all_projects)

#### Tasks

In [ ]:
all_tasks = fun.getTasks()
task_dict_id, task_dict_name = fun.getTasksDicts(all_tasks, projects_dict_id, projects_dict_name)

#### Sections

In [ ]:
all_sections = fun.getSections()
sections_dict_id, sections_dict_name = fun.getSectionsDicts(all_sections)

In [ ]:
sections_calen = fun.getSections(project_id = projects_dict_name["CALENDAR"])
sections_calen_dict_id, sections_calen_dict_name = fun.getSectionsDicts(sections_calen)

In [ ]:
sections_calen_dict_id

#### Test

In [ ]:
index = random.randint(0,len(all_tasks))
try:
    print(all_tasks[index]['content'], "-", all_tasks[index]['due']['date'])
except TypeError:
    print(all_tasks[index]['content'], "-", "No date")

#### Labels

In [ ]:
label_names=[]
for task in all_tasks:
    gross_labels=task['labels']
    for label in gross_labels:
        if label not in label_names:
            label_names.append(label)
label_names

#### F1 Calendar

In [ ]:
def getF1Calendar():
    return True

### `Orders`

#### Creating tasks

In [ ]:
def createTask(
    assignee_id = None,
    comment_count = 0,
    is_completed = False,
    content = "No se añadio título",
    description = None,
    due = None,
    duration = None,
    labels = [],
    order = 1,
    priority = 4,
    project_id = projects_dict_name['Inbox'],
    section_id = None,
    parent_id = None,
):
    for label in labels:
        if label not in label_names:
            return f'La etiqueta {label} no existe'
    try:
        task = api.add_task(
            assignee_id = assignee_id,
            comment_count = comment_count,
            is_completed = is_completed,
            content = content,
            description = description,
            due = due,
            duration = duration,
            labels = labels,
            order = order,
            priority = fun.priorityInversal(priority),
            project_id = project_id,
            section_id = section_id,
            parent_id = parent_id,
        )
        print(f'Task "{task.content}" ({task.id}) was created correctly')
    except Exception as error:
        print(error)

In [ ]:
createTask(content='prueba_buena',
            labels=[],
            priority=1,
            due={"string":"today"}
)
all_tasks = fun.getTasks()
task_dict_id, task_dict_name = fun.getTasksDicts(all_tasks, projects_dict_id, projects_dict_name)

#### Editing tasks

In [ ]:
def editTask(
    task_id,
    _content = None,
    _assignee_id = None,
    _comment_count = None,
    _is_completed = None,
    _description = None,
    _due = {},
    _labels = None,
    _order = None,
    _priority = None,
    _project_id = None,
    _section_id = None,
    _parent_id = None,
):
    task = fun.getTask(task_id)
    content = task.content if _content==None else _content
    assignee_id = task.assignee_id if _assignee_id==None else _assignee_id
    comment_count = task.comment_count if _comment_count==None else _comment_count
    is_completed = task.is_completed if _is_completed==None else _is_completed
    description = task.description if _description==None else _description
    due = task.due if _due=={None} else _due
    labels = _labels if _labels!=None else task.labels
    order = task.order if _order==None else None
    priority = task.priority if _priority==None else _priority
    project_id = task.project_id if _project_id==None else _project_id
    section_id = task.section_id if _section_id==None else _section_id
    parent_id = task.parent_id if _parent_id==None else _parent_id
    
    for label in labels:
        if label not in label_names:
            return f'La etiqueta {label} no existe'
    try:
        api.update_task(
            task_id = task_id,
            assignee_id = assignee_id,
            comment_count = comment_count,
            is_completed = is_completed,
            content = content,
            description = description,
            labels = labels,
            order = order,
            priority = priority,
            project_id = project_id,
            section_id = section_id,
            parent_id = parent_id,
            due = due
        )
        if _content != None:
            print(f'Task "{task.content}" updated correctly as "{_content}"')
        else:
            print(f'Task "{task.content}" updated correctly as "{task.content}"')
    except Exception as error:
        print(error)

In [ ]:
editTask(task_id = "7810532791", _description = "a ver esa poyita d marikon")
all_tasks = fun.getTasks()
task_dict_id, task_dict_name = fun.getTasksDicts(all_tasks, projects_dict_id, projects_dict_name)

#### Add duration label

In [ ]:
task_dict_name["LA molécula"]
all_tasks = fun.getTasks()
task_dict_id, task_dict_name = fun.getTasksDicts(all_tasks, projects_dict_id, projects_dict_name)

In [ ]:
def updateTaskDurationLabel(task_id, task_duration):
    duration_label = fun.getDurationLabel(task_duration)
    task_id = task_id[0]
    task_labels_without_duration = fun.getLabelsWithoutDuration(task_id)
    task_labels_without_duration.append(duration_label)
    editTask(task_id=task_id,
             _labels=task_labels_without_duration)

In [ ]:
all_tasks = fun.getTasks()
task_dict_id, task_dict_name = fun.getTasksDicts(all_tasks, projects_dict_id, projects_dict_name)
to_update = False

for task in all_tasks:
    if task['duration'] != None and task['due'] != None:
        if fun.getDurationLabel(task['duration']['amount']) not in task['labels']:
            to_update = True
            updateTaskDurationLabel([task['id']], task['duration']['amount'])

if not to_update:
    print('No task to be updated')
else:
    all_tasks = fun.getTasks()
    task_dict_id, task_dict_name = fun.getTasksDicts(all_tasks, projects_dict_id, projects_dict_name)


#### Capitalize titles

In [ ]:
task_id = '7810704210'
task = api.get_task(task_id = task_id)
print(task.content)
editTask(task_id = task_id,
         _content = (task.content)[0].upper() + (task.content)[1:])
all_tasks = fun.getTasks()
task_dict_id, task_dict_name = fun.getTasksDicts(all_tasks, projects_dict_id, projects_dict_name)

In [ ]:
alguna_cambiada = False
for tareita in all_tasks[101:]:
    # print(tareita["content"])
    if tareita["content"][0].upper() != tareita["content"][0]:
        alguna_cambiada = True
        task_id = tareita["id"]
        editTask(task_id = task_id,
            _content = tareita["content"][0].upper() + tareita["content"][1:])

if not alguna_cambiada:
    print("No tasks needed 2 b updated")

all_tasks = fun.getTasks()
task_dict_id, task_dict_name = fun.getTasksDicts(all_tasks, projects_dict_id, projects_dict_name)